# Brief Introduction to R and Descriptive Analysis of the Dataset

In this document, we will briefly introduce R and installing all the required packages for this course. A descriptive analysis of the dataset is then operated.

## Installing the required packages

The following commands will load the package if they are already installed. If they are not yet installed, they will be installed and loaded afterwards. Note that for Windows users, Rtools is required for some packages (e.g., CASdatasets). This list may not be exhaustive and other packages may be required in other notebooks.

In [ ]:
if (!require("xts")) install.packages("xts")
if (!require("sp")) install.packages("sp")
if (!require("CASdatasets")) install.packages("CASdatasets", repos = "http://cas.uqam.ca/pub/", type="source")
if (!require("caret")) install.packages("caret")
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("mgcv")) install.packages("mgcv")
if (!require("dplyr")) install.packages("dplyr")
if (!require("gridExtra")) install.packages("gridExtra")
if (!require("visreg")) install.packages("visreg")
if (!require("MASS")) install.packages("MASS")
if (!require("plotrix")) install.packages("plotrix")
if (!require("rgeos")) install.packages("rgeos")
if (!require("rgdal")) install.packages("rgdal")
if (!require("xtable")) install.packages("xtable")
if (!require("maptools")) install.packages("maptools")
if (!require("scales")) install.packages("scales")
if (!require("broom")) install.packages("broom")
if (!require("stringi")) install.packages("stringi")
if (!require("arrow")) install.packages("arrow")


require("CASdatasets")
require("ggplot2")
require("mgcv")
require("caret")
require("gridExtra")
require("dplyr")
require("visreg")
require("MASS")
require("plotrix")
require("rgdal")
require("rgeos")
require("xtable")
require("maptools")
require("scales")
require("broom")
require("stringi")
require("arrow");

In this jupyter notebook, we will use the following options to set the width and height of our plots.

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4, repr.plot.res = 250)

## Getting started with the dataset
### Loading the dataset

We will now load a dataset from the **CASdatasets** package. In case you were not able to install the **CASdatasets** package, we also provide a parquet file of the dataset (see more on that below).

We can simply load the dataset with the following command:

In [ ]:
data("freMTPLfreq")

To keep it simple and illustrative, we will only keep a subset of this dataset. Each line corresponds to a policy. We will restrict ourself to the policies covering a vehicle aged between 0 and 25 years. Also, we will only keep policies that were covered for a maximum of one year.

We will use the **tidyverse** univers in this course, as it can be easier to read (and writing clear code is important!). Subsetting can be done with the **filter** function.

In [ ]:
dataset = freMTPLfreq %>% filter(Exposure<=1 & Exposure >= 0 & CarAge<=25)

Note the *pipe* operator which allows to chain operations. We could also have written the following. We check that we obtain the same result with the **all.equal(dataset, dataset_alternative)** function. To save some memory we then remove the alternative dataset.

In [ ]:
dataset_alternative = freMTPLfreq %>% 
                                filter(Exposure <= 1) %>% 
                                filter(Exposure >= 0) %>% 
                                filter(CarAge<=25)

sprintf("Are the two datasets equal ? %s", ifelse(all.equal(dataset, dataset_alternative), "Yes", "No"))
rm(dataset_alternative)

In [ ]:
write_parquet(dataset, sink = "../data/dataset.parquet", compression = "gzip")

We will save the dataset into a *parquet* file, so we don't need to load the CASdatasets package anymore and filter the data.

For those that could not install this package, now is the time to load the provided *parquet* file.

In [ ]:
dataset = read_parquet(file = "../data/dataset.parquet")

### Checking the dataset

We can check that the dataset is correctly loaded with the following functions.A good idea is to check whether the dataset has been loaded correctly. To do this, the following tools can be used:

 - **head** allows to visualize the first 6 lines of the dataset.

In [ ]:
head(dataset)

- **str** allows to see the format of the different variables. We will typically distinguish numerical variables (real numbers or integers) and factors (categorical data).

In [ ]:
str(dataset)

- **summary** allows to compute for each variable some summary statistics.

In [ ]:
summary(dataset)

If one needs some **help** on a function, typing a question mark and the name of the function in the console opens the help file of the function. For instance,

In [ ]:
?head

## Descriptive Analysis of the dataset

We will now proceed with a descriptive analysis of this dataset. We will now have a descriptive analysis of the portfolio. The different variables available are

In [ ]:
names(dataset)

### PolicyID
The variable *PolicyID* related to a unique identifier of the policy. We can check that every policy appears only once in the dataset

In [ ]:
length(unique(dataset$PolicyID)) == nrow(dataset)

Another possibility is to check the frequency of each *PolicyID* using the function *table*. The result is a table that shows for each *PolicyID* the corresponding number of lines in the dataset. We can then use a second time the function *table* in this result to show the frequency. We expect to have only **ones** (with possibily zeros), meaning each *PolicyID* has a unique line.

In [ ]:
table(table(dataset$PolicyID))

**To what corresponds the 0 ?** 

It appears that in this dataset the variable *PolicyID* is a **factor**. A factor variable has different *levels*. It appears that some PolicyID may be missing here (removed from the dataset, probably when we filtered out some policies). It is as if we had a 3-level categorical variable, for instance, color of a car, which takes three possible values: red, blue, gray, but in our dataset, we would only have red and blue cars. Gray would still be a level, but with no observation (i.e. no row) corresponding to a gray car.

To remove unused levels, we can use on the function **droplevels**.

### Exposure in month
The Exposure reveals the fraction of the year during which the policyholder is in the portfolio. 
We can compute the total exposure by summing the policyholders' exposures. 
Here we find:

In [ ]:
sprintf("%s years",scales::number(sum(dataset$Exposure), accuracy = 0.1))

We can show the number of months of exposure on a table. The function *cut* allows to categorize (bin) a numerical variable. We can specify where to 'break' and give a name to each level using the *labels* argument. The output is a factor variable.

In [ ]:
table_exposures = table(cut(dataset$Exposure, breaks = seq(from = 0, to = 1,by = 1/12), labels = 1:12))
table_exposures

Using the function *prop.table*, it is possible to represent this information in relative terms show the number of months of exposure on a table.

In [ ]:
Exposures_prop = prop.table(table_exposures)
round(Exposures_prop, 4)*100

Alternatively, we can use a barplot, using **ggplot2** !

In [ ]:
ggplot(dataset)+geom_bar(aes(x=cut(Exposure,
                                   breaks = seq(from = 0, to = 1,by = 1/12), labels = 1:12))) + 
  scale_x_discrete(name = "Number of months") + 
  scale_y_continuous(name = "Number of Policies", label = label_number()) + 
  ggtitle("Exposure in months")

What if we also want to show the percentage on the bars ?

In [ ]:
ggplot(dataset, aes(x=cut(Exposure, breaks = seq(from = 0, to = 1,by = 1/12), labels = 1:12), 
                    label = scales::percent(prop.table(stat(count)), accuracy = 0.1)))+
  geom_bar() + 
  geom_text(stat = 'count',
            vjust = -0.5,
            size = 3) +
  scale_x_discrete(name = "Number of months") + 
  scale_y_continuous(name = "Number of Policies", 
                     label = label_number()) + 
  ggtitle("Exposure in months")

Note that a barplot is used to plot factor variables (categorical variables). In our case, we categorized the variable Exposure using the function *cut*. If we do not want to categorize this variable, we should use a histogram. We can specify the number of bins (= 12) or the binwidth (= 1/12).

In [ ]:
ggplot(dataset, aes(x=Exposure))+geom_histogram(binwidth =1/12, fill='gray', color='white') +
  scale_x_continuous(name = "Exposure in fraction of years", breaks=seq(0,1,1/12), labels = round(seq(0,1,1/12), 3))+
  scale_y_continuous(name = 'Number of Polices', labels = label_number()) + 
  ggtitle("Exposure in fraction of years")

If you are not familiar with ggplot, I could recommend this cheat-sheet: https://github.com/rstudio/cheatsheets/blob/main/data-visualization-2.1.pdf

### Number of claim : ClaimNb


In [ ]:
ggplot(dataset, aes(x=ClaimNb))+
  geom_bar()+
  geom_label(stat='count', 
             aes(label =  percent(prop.table(after_stat(count)), 
                                     accuracy = 0.01)),
             vjust = 0.5)+
  scale_x_continuous(name = "Number of Claims")+
  scale_y_continuous(name = "Number of Polices", 
                     labels = label_number())+
  ggtitle("Proportion of policies by number of claims")

We can compute the average claim frequency in this portfolio, taking into account the different exposures.

In [ ]:
scales::percent(sum(dataset$ClaimNb) / sum(dataset$Exposure), accuracy = 0.01)

Let us now look at the other variables.

### Power

The variable **Power** is a categorized variable, related to the power of the car. The levels of the variable are ordered categorically.
We can see the different **levels** of a **factor** by using the function *level* in R:

In [ ]:
levels(dataset$Power)

In [ ]:
table(dataset$Power)

Remember however, that in insurance, exposures may differ from one policyholder to another. Hence, the table above, does NOT measure the exposure in each level of the variable *Power*. We can use the functions *group_by* and *summarise* from package **dplyr** to give us the exposure in each level of the variable.

Check out the cheatsheet https://github.com/rstudio/cheatsheets/blob/main/data-transformation.pdf

In [ ]:
Power.summary = dataset %>% group_by(Power) %>% summarise(totalExposure = sum(Exposure),
                                                          Number.Observations = length(Exposure))
Power.summary

We can show this on a plot as well:

In [ ]:
ggplot(Power.summary, aes(x=Power, 
                          y=totalExposure, 
                          fill=Power, 
                          color=Power, 
                          label=scales::number(totalExposure))) + 
  geom_bar(stat="identity")+
  geom_text(stat='identity',  vjust=-0.5)+
  scale_y_continuous(name = "Exposure in years", labels = scales::number)+
  scale_colour_discrete(guide = "none")+
  scale_fill_discrete(guide="none")

Let us now look at the observed claim frequency in each level

In [ ]:
Power.summary = dataset %>% group_by(Power) %>% summarise(totalExposure = sum(Exposure),
                                                          Number.Observations = length(Exposure),
                                                          Number.Claims = sum(ClaimNb),
                                                          Obs.Claim.Frequency = sum(ClaimNb)/sum(Exposure))
Power.summary

We can compute the ratio to the portfolio claim frequency and plot the claim frequencies.

In [ ]:
portfolio.cf = sum(dataset$ClaimNb)/ sum(dataset$Exposure)
# Can also be written as
portfolio.cf = with(dataset, sum(ClaimNb)/sum(Exposure))

ggplot(Power.summary, aes(x=Power, 
                          y=Obs.Claim.Frequency,
                          color=Obs.Claim.Frequency,
                          fill=Obs.Claim.Frequency,
                          label = percent(Obs.Claim.Frequency, accuracy = 0.01))) + 
  geom_bar(stat='identity') + 
  geom_hline(aes(yintercept=portfolio.cf), color="black", size= 2, linetype="dashed", alpha = 0.33) +
  geom_label(vjust=-0.21, fill="white", alpha = 0.25)+
  annotate(geom="text", 
           x='m', y=portfolio.cf, 
           vjust=-0.5, 
           label=paste("Average claim freq. of portfolio: ", percent(portfolio.cf, accuracy = 0.01)), 
           color="black")+
  scale_y_continuous(name = "Observed Claim Frequency", labels = percent_format(accuracy = 0.01))+
  theme(legend.position = 'none')

### CarAge

The vehicle age, in years. This is the first continuous variable that we encounter (although it only takes discrete values).

In [ ]:
ggplot(dataset, 
       aes(x=CarAge)) + geom_bar()  + 
  scale_x_continuous(name = "Age of the Car", breaks=seq(0,100,5))+
  scale_y_continuous(name = "Number of Polices", labels=label_number())

Alternatively, we can use a histogram, with a binwidth of 1.

In [ ]:
ggplot(dataset, 
       aes(x=CarAge)) + geom_histogram(binwidth = 1, color = 'black', fill='white')  + 
  scale_x_continuous(name = "Age of the Car", breaks=seq(0,100,5))+
  scale_y_continuous(name = "Number of Polices", labels=label_number())

Again, here, the exposures are not considered on the barplot/histogram. We can use **ddply** to correct this.

In [ ]:
CarAge.summary = dataset %>% group_by(CarAge) %>% summarise(totalExposure = sum(Exposure),
                                                            Number.Observations = length(Exposure))
CarAge.summary

Then, we can plot the data onto a barplot, as before.

In [ ]:
ggplot(CarAge.summary, aes(x=CarAge,
                          y=totalExposure, 
                          fill=factor(CarAge), 
                          color=factor(CarAge), 
                          label=label_number(accuracy=1)(totalExposure))) + 
  geom_bar(stat="identity")+
  geom_text(stat='identity', color="black", hjust=0.25, vjust=0.5,  angle=45, check_overlap=TRUE)+
  scale_x_continuous(breaks = seq(0,100,5))+
  scale_y_continuous(name = "Exposure in years", labels = label_number())+
  theme(legend.position = 'none')

We can see a large difference, specially for new cars, which makes sense ! Indeed, let us look at the Exposure for recent vehicles, using a boxplot for instance.

In [ ]:
ggplot(dataset%>% filter(CarAge<5), aes(x=CarAge, y=Exposure,  group=CarAge)) + 
  geom_boxplot() +
  ggtitle("Exposure of recent cars")

Let us now also compute the claim frequencies by age of car and plot them.

In [ ]:
CarAge.summary = dataset %>% group_by(CarAge) %>% summarise(totalExposure = sum(Exposure), 
                                                             Number.Observations = length(Exposure), 
                                                             Number.Claims = sum(ClaimNb), 
                                                             Obs.Claim.Freq = sum(ClaimNb)/sum(Exposure))

portfolio.cf = with(dataset, sum(ClaimNb)/ sum(Exposure))

ggplot(CarAge.summary, aes(x=CarAge, 
                          y=Obs.Claim.Freq,
                          label = percent(Obs.Claim.Freq, accuracy = 0.01))) + 
  geom_point() + geom_line()+
  geom_hline(yintercept=portfolio.cf, color="black", size= 2, linetype="dashed", alpha = 0.33) +
    annotate(geom="text", 
           x=20, y=portfolio.cf, 
           vjust=-0.5, 
           label=paste("Average claim freq. of portfolio: ", percent(portfolio.cf, accuracy = 0.01)), 
           color="black")+
  scale_x_continuous(name = "Age of the Car", breaks=seq(0,100,5))+
  scale_y_continuous(name = "Observed Claim Frequency", labels = percent_format(accuracy = 0.01))+
  theme(legend.position = 'none')


### DriverAge
Similarly to the Age of the Car, we can visualize the Age of the Drivers.

In [ ]:
DriverAge.summary = dataset %>% group_by(DriverAge) %>% summarise(totalExposure = sum(Exposure), 
                                                                  Number.Observations = length(Exposure), 
                                                                  Number.Claims = sum(ClaimNb), 
                                                                  Obs.Claim.Freq = sum(ClaimNb)/sum(Exposure))
head(DriverAge.summary,9)

We can show the Exposures by Age of the Driver...

In [ ]:
ggplot(DriverAge.summary, aes(x=DriverAge, y=totalExposure)) + 
  geom_bar(stat='identity', width=0.8) + 
  scale_y_continuous(name="Exposure in years", labels = label_number())+
  scale_x_continuous(name="Age of the Driver", breaks = seq(10,150,10))

... and the observed claim frequency by Age.

In [ ]:
ggplot(DriverAge.summary, aes(x=DriverAge, y=Obs.Claim.Freq)) + 
  geom_line()+geom_point()+
  scale_y_continuous(name="Observed Claim Frequency", labels = percent, breaks = seq(0,0.50,0.05))+
  scale_x_continuous(name="Age of the Driver", breaks = seq(10,150,10))

### Brand
The variable **Brand** is a categorized variable, related to the brand of the car. 
We can see the different *levels* of a *factor* by using the function **level** in R:

In [ ]:
levels(dataset$Brand)

In [ ]:
Brand.summary = dataset %>% group_by(Brand) %>% summarise(totalExposure = sum(Exposure), 
                                                          Number.Observations = length(Exposure), 
                                                          Number.Claims = sum(ClaimNb), 
                                                          Obs.Claim.Freq = sum(ClaimNb)/sum(Exposure))
                                          
Brand.summary

In [ ]:
ggplot(Brand.summary, aes(x=reorder(Brand,totalExposure), 
                          y=totalExposure, 
                          fill=Brand,
                          label = label_number()(totalExposure))) +
  geom_bar(stat='identity') +
  coord_flip()+guides(fill='none')+
  scale_x_discrete(name = "") + 
  scale_y_continuous("Exposure in years", labels = label_number(), expand = c(0.10, 0))+
  geom_label()

Let us now look at the claim frequency by Brand of the car.

In [ ]:
ggplot(Brand.summary, aes(x=reorder(Brand,Obs.Claim.Freq), 
                          y=Obs.Claim.Freq, 
                          fill=Brand,
                          label = percent(Obs.Claim.Freq, accuracy=0.1))) +
  geom_bar(stat='identity') +
  geom_label(hjust=+1.2)+
  coord_flip()+guides(fill='none')+ ggtitle("Observed Claim Frequencies by Brand of the car")+
  scale_x_discrete(name = "Brand") + 
  scale_y_continuous("Observed claim Frequency", labels = percent)

### Gas
The variable *Gas* is a categorized variable, related to the fuel of the car. 
We can see the different *levels* of a *factor* by using the function **level** in R:

In [ ]:
levels(dataset$Gas)

In [ ]:
Gas.summary = dataset %>% group_by(Gas) %>% summarise(totalExposure = sum(Exposure), 
                                                      Number.Observations = length(Exposure), 
                                                      Number.Claims = sum(ClaimNb), 
                                                      Obs.Claim.Freq = sum(ClaimNb)/sum(Exposure))
ggplot(Gas.summary, aes(x=Gas, y=totalExposure, fill=Gas, label = number(totalExposure))) + 
  geom_bar(stat="identity") + 
  geom_label()+
  guides(fill='none')+
  scale_x_discrete(name = "Fuel")+
  scale_y_continuous(name='Total Exposure (in years)', labels = number)

There seems to be a similar amount of Diesel and Regular gas vehicles in the portfolio.
It is generally expected that Diesel have a higher claim frequency. Does this also hold on our dataset ?

In [ ]:
ggplot(Gas.summary, 
       aes(x=Gas, y=Obs.Claim.Freq, 
           fill=Gas, 
           label = percent(Obs.Claim.Freq))) + 
  geom_bar(stat="identity") + 
  geom_label()+  guides(fill='none')+
  scale_x_discrete(name = "Fuel") + 
  scale_y_continuous("Observed claim Frequency", labels = percent)

### Region
The variable *Region* is a categorized variable, related to the region of the place of residence. 
We can see the different *levels* of a *factor* by using the function **level** in R:

In [ ]:
levels(dataset$Region)

What are the Exposures in each region ? What are the observed claim frequencies ?

In [ ]:
Region.summary = dataset %>% group_by(Region) %>% summarize(totalExposure = sum(Exposure), 
                                                           Number.Observations = length(Exposure), 
                                                           Number.Claims = sum(ClaimNb), 
                                                           Obs.Claim.Freq = sum(ClaimNb)/sum(Exposure))
Region.summary

#### Creating Maps

We can plot a map with the observed claim frequencies and the total Exposure.
We first need to obtain the shape files (which contain the borders of each administrative area.)

1. Download shapefile from  http://www.diva-gis.org/gData
2. Extract all the files from the zip files, in a directory called shapefiles in your working directory

In [ ]:
area <- rgdal::readOGR("shapefiles/FRA_adm1.shp", use_iconv = TRUE, encoding = "UTF-8") # From http://www.diva-gis.org/gData

# Note that the tidy function will remove the data.
area_tidy = tidy(area) # package broom

#Plot an "empty" map
ggplot(area_tidy, aes(x = long, y = lat, group = group)) +
  geom_polygon(color = "black", size = 0.1, fill = "lightgrey") +
  coord_equal() + theme_void()

We are now going to include our data into the map

In [ ]:
#First we re-include the data (that disappeared with the tidy function)
area$id <- row.names(area)
area_tidy2 <- area_tidy %>% full_join(area@data, by="id")

# Because of accents ...
area_tidy2$NAME_1 = stri_trans_general(str = area_tidy2$NAME_1, 
                   id = "Latin-ASCII")

# Which computed data do we want ?
data_to_add = Region.summary[,c("Region", "totalExposure", "Obs.Claim.Freq")]
# Merge it 
area_tidy2 <- area_tidy2 %>% full_join(data_to_add, by = c("NAME_1" = "Region"))

# Very important: Do not forget to sort by "order" variable.
# area_tidy2 = area_tidy2[order(area_tidy2$order),]
# Easier with dplyr:
area_tidy2 = area_tidy2 %>% arrange(order)

# Finally, plot the data: the claim frequencies
p1 = ggplot(area_tidy2, aes(long, lat, group=group, fill = Obs.Claim.Freq)) + ggtitle("Observed Claim Frequencies")+
  geom_polygon(color="black")+
  scale_fill_gradient(low = "green", high = "red", name="Obs. Claim Freq.")+
xlab("Longitude") + ylab("Latitude") + theme_void()

# and the exposures (on a log-scale)...
p2 = ggplot(area_tidy2, aes(long, lat, group=group, fill = log(totalExposure))) + ggtitle("log Exposures in years")+
  geom_polygon(color="black")+
  scale_fill_gradient(low = "green", high = "red", name="log Exposure")+
xlab("Longitude") + ylab("Latitude") + theme_void()


# Show them next to each other
gridExtra::grid.arrange(p1, p2, ncol=2)

### Density
The Density represents here the density of the population at the place of residence.
Let us take a look at the densities in the dataset.


In [ ]:
summary(dataset$Density)
ggplot(dataset, aes(Density)) + geom_histogram(bins=200)

Here, contrary to the age of the driver, or the age of the car, the density has lots of different values, as we can see below.

In [ ]:
length(unique(dataset$Density))

Let us still compute as before the summary statistics and plot them ...

In [ ]:
Density.summary = dataset %>% group_by(Density) %>%
                   summarise(totalExposure = sum(Exposure), 
                                Number.Observations = length(Exposure), 
                                Number.Claims = sum(ClaimNb), 
                                Obs.Claim.Freq = sum(ClaimNb)/sum(Exposure))
ggplot(Density.summary, aes(x=Density, y=Obs.Claim.Freq)) + geom_point()

... but realize it is impossible to see a trend. One way out is to categorize the variable. We will see later (GAM) that it is possible to estimate a smooth function, which avoid the arbitrary categorization.

We can categorize the variable using the function *cut*.

In [ ]:
dataset$DensityCAT = cut(dataset$Density, breaks = quantile(dataset$Density, probs = seq(from = 0, to = 1, by=0.1)),
                         include.lowest = TRUE)
table(dataset$DensityCAT)
levels(dataset$DensityCAT) <- LETTERS[1:10]

Then, we can apply the same strategy as above.

In [ ]:
Density.summary = dataset %>% group_by(DensityCAT) %>%
                  summarise(totalExposure = sum(Exposure), 
                            Number.Observations = length(Exposure), 
                            Number.Claims = sum(ClaimNb), 
                            Obs.Claim.Freq = sum(ClaimNb)/sum(Exposure))

ggplot(Density.summary, 
       aes(x=DensityCAT, 
           y=Obs.Claim.Freq, 
           fill=DensityCAT, 
           label = percent(Obs.Claim.Freq))) + 
  geom_bar(stat="identity") + 
  geom_label()+  guides(fill='none')+
  scale_x_discrete(name = "Density") + 
  scale_y_continuous("Observed claim Frequency", labels = percent)

## Interactions

We can of course also dive into some interactions. For instance, we could analyse the effect of the car Age combined with the Fuel (Gas).

### Fuel and Car Age

In [ ]:
CarAge.Fuel.summary = dataset %>% group_by(CarAge, Gas) %>% 
                      summarise(totalExposure = sum(Exposure),
                                Number.Observations = length(Exposure),
                                Number.Claims = sum(ClaimNb),
                                Obs.Claim.Freq = sum(ClaimNb)/sum(Exposure))

ggplot(CarAge.Fuel.summary, aes(x=CarAge, 
                                y=Obs.Claim.Freq)) + 
  facet_wrap(~Gas)+
  geom_bar(stat="identity") + 
  scale_x_continuous(name = "Age of the Car", breaks=seq(0,100,5))+
  scale_y_continuous(name = "Observed Claim Frequency", labels = percent_format(accuracy = 0.01))+
  theme(legend.position = 'none')

### Fuel and Driver Age

We will illustrate another way to show this kind of data, by overlapping both bars.

In [ ]:
DriverAge.Fuel.summary = dataset %>% group_by(DriverAge, Gas) %>% 
                        summarize(Obs.Claim.Freq = sum(ClaimNb)/sum(Exposure))

ggplot(data=DriverAge.Fuel.summary, aes(x=DriverAge, 
                                        y=Obs.Claim.Freq, 
                                        fill=Gas, 
                                        color=Gas, 
                                        alpha=Gas)) +
  geom_bar(stat="identity", position ="identity") +
  scale_x_continuous(name = "Age of the Driver", breaks = seq(0,100,5))+
  scale_y_continuous(name = "Observed Claim Frequency", labels = label_percent())+
  scale_colour_manual(values=c("lightblue4", "red")) +
  scale_fill_manual(values=c("lightblue", "pink")) +
  scale_alpha_manual(values=c(.3, .8))+
  theme_bw()

# Useful Links

- https://github.com/rstudio/cheatsheets/blob/main/data-transformation.pdf
- https://github.com/rstudio/cheatsheets/blob/main/data-visualization-2.1.pdf